In [1]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
import inspect

### Relevante Parameter:

array_phi = 1 / (1 + np.exp(delta_0[state] + delta_1[state] * np.log(LY_monat)))

array_lambdaY = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))

    if r[0] > 0:
        for i in range(len(y_monat)):
            for j in range(y_monat[i] + 1):
                if j == 0:
                    pmf = phi[i] + (1 - phi[i]) * (1 + lambdaY[i] / r[0])**(-r[0])
                elif j > 0:
                    numerator = math.gamma(j + r[0])
                    denominator = math.factorial(j) * math.gamma(r[0])
                    gamma_term = numerator / denominator
                    pmf= (1 - phi[i]) * gamma_term * (1 + lambdaY[i] / r[0])**(-r[0]) * (1 + r[0] / lambdaY[i])**(-j)
                cdf[i] += pmf
    return cdf

Die Öffungszahlen sind mit einer ZINBD modelliert und die Werte folgender Tabellen haben Einfluss:

1. df_LY
2. LO_monat
3. df_trans_werte
4. df_HY

folgende Parameter haben einen Einfluss:
1. delta_0 (1 x anzahl_states array, also state abhängig)
2. delta_1 (1 x anzahl_states array, also state abhängig)

3. alpha_p (1 x anzahl_states array, also state abhängig)
4. beta_p_1 (1 x anzahl_states array, also state abhängig)
5. beta_p_1 (1 x anzahl_states array, also state abhängig)



(1 x 1 array, state unabhängig)

In [2]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde.csv'

df_LY = pd.read_csv(os.path.join(path, filename))

In [3]:
# LO monat
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

In [4]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde.csv'

df_trans_werte = pd.read_csv(os.path.join(path, filename))

In [5]:
# EM_monat aus der HY Zahlen der versendeten Mails ziehen
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

### Datenanpassung wie bei der Berechnung der optimalen Parameter

In [6]:
## Alle DataFrame Werte von Float zu int
df_LY = df_LY.astype(int)
df_LO = df_LO.astype(int)
df_trans_werte = df_HY.astype(int)
df_HY = df_LY.astype(int)

### Sicherstellen dass alle df dieselben Kunden verwenden

In [7]:
# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LY = df_LY[df_LY['iid'].isin(df_LO['iid'])]
df_LO = df_LO[df_LO['iid'].isin(df_trans_werte['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_HY['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LY['iid'])]

In [8]:
len(df_LY)

647

In [9]:
len(df_LO)

647

In [10]:
len(df_trans_werte)

697

In [11]:
len(df_HY)

647

In [12]:
# Siherstellen dass alle dataframes die Kunden in derselben Reihenfolge auflisten
df_LY = df_LY.sort_values('iid')
df_LO = df_LO.sort_values('iid')
df_trans_werte = df_trans_werte.sort_values('iid')
df_HY = df_HY.sort_values('iid')

#Index resetten
df_LY = df_LY.reset_index(drop = True)
df_LO = df_LO.reset_index(drop = True)
df_trans_werte = df_trans_werte.reset_index(drop = True)
df_HY = df_HY.reset_index(drop = True)

### Ausreißer bei den Kunden löschen (wie bei den Berechnungen)

In [13]:
kunden_pos = [443, 387, 408, 415, 451, 531, 364, 351, 348, 501, 408, 285, 422, 251, 220]

In [14]:
# Zeilen löschen
df_LY = df_LY.drop(kunden_pos).reset_index(drop = True)
df_LO = df_LO.drop(kunden_pos).reset_index(drop = True)
df_trans_werte = df_trans_werte.drop(kunden_pos).reset_index(drop = True)
df_HY = df_HY.drop(kunden_pos).reset_index(drop = True)

### Spezifikation des Modells

In [15]:
anzahl_kunden = len(df_LY["iid"])
anzahl_kunden

633

In [16]:
anzahl_monate = len(df_LY.columns[df_LY.columns != 'iid'])
anzahl_monate

15

In [17]:
anzahl_states = 3

### Nelder-Mead mit poisite Wahresceinlichkeiten nach state 1, 30k Berechungen

### 1. Herauslesen der Parameter aus dem array der gesamten geschätzten Parameter

In [20]:
estimated_params=[ -8.20399103e-01,  4.47364379e+01,  6.55459562e+01, -7.12664184e-01,
       -2.48373911e+02,  4.63946330e+02, -1.89923145e+00, -5.94099736e+01,
        7.59497882e+01,  5.18311697e-01, -2.61490366e-01,  8.93093831e-01,
        1.63703041e-01,  7.28623716e-03, -2.62212305e-01,  9.68519753e-02,
        8.66641236e-02,  1.82355786e-01,  2.36506775e-01, -8.98038162e-02,
       -4.25975327e-01,  4.99874934e-01, -1.43621259e-01,  8.11571266e-01,
        1.42871394e+00, -7.33210161e-01,  2.96600857e-01,  1.65401014e-01,
        1.01422490e-02, -3.14895811e-01, -2.02066317e-01, -5.13156402e-01,
        3.44435091e-01,  1.23401987e+00, -1.31647262e+00, -1.48696412e+00,
       -6.23244984e-01,  1.30427990e+00, -6.95618496e-01, -5.10706288e-01,
        4.85842039e-01, -5.54033892e-01,  8.96741073e-02,  9.01273778e-02,
       -3.67011581e-01, -2.31574553e-02, -6.96592721e-01,  1.16671348e+00,
       -1.26661223e+00,  1.23556052e+02, -2.55630320e+01, -4.60157041e+00,
        4.69496096e-01, -4.73224796e-01,  1.61440816e-01, -7.12740205e-01,
       -1.95849638e+01, -1.30792666e-01,  2.79970169e-01, -2.96108429e-01,
        1.53815838e+00,  1.00040668e-01, -1.41070531e-02, -4.57917675e-01,
        1.49032916e+00]

In [21]:
alpha_size = anzahl_states**2
gamma_size = anzahl_states**2 * 4

alpha = estimated_params[:alpha_size]
gamma = estimated_params[alpha_size:alpha_size+gamma_size]
alpha_o = estimated_params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
beta_o = estimated_params[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
delta_0 = estimated_params[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
delta_1 = estimated_params[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
alpha_p = estimated_params[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
beta_p_1 = estimated_params[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
beta_p_2 = estimated_params[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
theta = estimated_params[alpha_size+gamma_size+anzahl_states*6+1:]

In [22]:
delta_0

[123.556052, -25.563032, -4.60157041]

In [23]:
delta_1

[0.469496096, -0.473224796, 0.161440816]

In [24]:
alpha_p

[-0.712740205, -19.5849638, -0.130792666]

In [25]:
beta_p_1

[0.279970169, -0.296108429, 1.53815838]

In [26]:
beta_p_2

[0.100040668, -0.0141070531, -0.457917675]

### Erstellen der erwarteten Käufe in den jeweiligen Monaten

In [28]:
# Erwartungswert = [r * (1-p)] / p

#### r, die Anzahl der Erfolge bis zum Abbruch, wurde vorher geschätzt

In [29]:
r = 0.779

#### Berechnun von p

In [27]:
def p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    EM_monat = df_HY[str_monat]
    LY_monat = df_LY[str_monat]
    
    array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat) +\
                      + delta_1[state] * np.log(LY_monat))
    array_p = array_exp / (1+ array_exp)
    
    return array_p

#### Berechnung der Erwartungswerte

In [30]:
# Speichern der Erwartungswerte in einem array
erwartung = np.zeros((anzahl_kunden, anzahl_states, anzahl_monate))

In [31]:
for state in range(1, anzahl_states + 1):
        for monat in range(1, anzahl_monate +1):
            p_array = p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
            erwartung[:, state-1, monat-1] = (r * (1-p_array) / p_array)

In [32]:
np.sum(erwartung[:,0,:])/erwartung[:,0,:].size

0.5105094319603598

In [33]:
np.sum(erwartung[:,1,:])/erwartung[:,1,:].size

8906312242.160769

In [34]:
np.sum(erwartung[:,2,:])/erwartung[:,2,:].size

1.8092770055696925e+26